In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

import kraft

In [3]:
data_directory_path = "/home/kwat/garden/data/cancer_cell_line/copy_number/"

kraft.path(data_directory_path)

In [ ]:
gene_x_cell_line = pd.read_csv("{}CCLE_gene_cn.csv".format(data_directory_path), index_col=0).T

gene_x_cell_line

In [ ]:
%timeit gene_x_cell_line.index = tuple(gene.split()[0] for gene in gene_x_cell_line.index.values)

In [ ]:
%timeit a.size

In [ ]:
%timeit "a"

In [ ]:
%timeit 1

In [ ]:
ayalakkkttwpppppppppppppppppp

In [ ]:
%timeit gene_x_cell_line.index = tuple(gene.split()[0] for gene in gene_x_cell_line.index.to_numpy()aaaaaaarrrrrrraaaaaaayyyyyyaaaaaaaaiiiiiilllllaaaa)

In [ ]:
gene_x_cell_line.columns = kraft.normalize_cell_line_names(gene_x_cell_line.columns.values)

In [ ]:
kraft.error_axis(gene_x_cell_line)

gene_x_cell_line.index.name = "Gene"

gene_x_cell_line.to_csv("copy_number.gene_x_cell_line.tsv", sep="\t")

gene_x_cell_line

In [ ]:
mean = np.nanmean(gene_x_cell_line.to_numpy())

std = np.nanstd(gene_x_cell_line.to_numpy())

threshold_loss = mean - 3 * std

threshold_gain = mean + 3 * std

threshold_loss, threshold_gain

In [ ]:
%timeit a.values

In [ ]:
%timeit a.to_numpy()

In [ ]:
def split_copy_numbers(copy_numbers, threshold_loss, threshold_gain):

    vector = np.full(copy_numbers.size, np.nan)

    vector[copy_numbers.values < threshold_loss] = -1

    vector[threshold_gain < copy_numbers.values] = 1

    gene = copy_numbers.name

    dataframe = kraft.binarize(pd.Series(vector, index=copy_numbers.index, name=gene))

    dataframe = dataframe.loc[dataframe.index.dropna()]

    dataframe.index = dataframe.index.map(
        {-1: "{} -".format(gene), 1: "{} +".format(gene)}
    )

    return dataframe

In [ ]:
def split_copy_numbers(copy_numbers, threshold_loss, threshold_gain):
    
    gene_loss_gain_x_cell_line = np.full((2, copy_numbers.size), np.nan)

    for i, number in enumerate(copy_numbers):
        
        if number < threshold_loss:
            
            gene_loss_gain_x_cell_line[0, number] = 1
            
        elif threshold_loss < number:
            
            gene_loss_gain_x_cell_line[1, number] = 1

    return pd.DataFrame(gene_loss_gain_x_cell_line, index=("{} loss".format(), "{} gain".format()), columns=copy_numbers.index)

In [ ]:
cell_line_numbers = gene_x_cell_line.iloc[100, :]

In [ ]:
split_loss_gain(cell_line_numbers, threshold_loss, threshold_gain).sum(axis=1)

In [ ]:
%timeit split_loss_gain(cell_line_numbers, threshold_loss, threshold_gain)

In [ ]:
def split_loss_gain2(cell_line_copy_numbers, threshold_loss, threshold_gain):
    
    cell_line_copy_numbers = cell_line_copy_numbers.copy()
    
    is_loss = cell_line_copy_numbers.values < threshold_loss
    
    is_gain = threshold_gain < cell_line_copy_numbers.values
    
    cell_line_copy_numbers.values[is_loss] = -1
    
    cell_line_copy_numbers.values[is_gain] = 1
    
    cell_line_copy_numbers[~(is_loss | is_gain)] = np.nan
    
    loss_gain_x_cell_line = kraft.binarize(cell_line_copy_numbers)
    
    gene = cell_line_copy_numbers.name
    
    loss_gain_x_cell_line.index = loss_gain_x_cell_line.index.map({
        -1: "{} loss".format(gene),
        1: "{} gain".format(gene),
    })

    loss_gain_x_cell_line = loss_gain_x_cell_line.loc[loss_gain_x_cell_line.index.dropna(), :]

    return loss_gain_x_cell_line

In [ ]:
split_loss_gain2(cell_line_numbers, threshold_loss, threshold_gain).sum(axis=1)

In [ ]:
%timeit split_loss_gain2(cell_line_numbers, threshold_loss, threshold_gain)

In [ ]:
file_path = "gene_loss_gain_x_cell_line.tsv"

n = gene_x_cell_line.shape[0]

n_per_print = n // 100

with open(file_path, mode="w") as io:

    for i, (gene, copy_numbers) in enumerate(gene_x_cell_line.iterrows()):

        if i % n_per_print == 0:

            print("{}/{} ...".format(i + 1, n))

        split_copy_numbers(copy_numbers, threshold_loss, threshold_gain).to_csv(
            io, sep="\t", header=i == 0, mode="a"
        )

gene_loss_gain_x_cell_line = pd.read_csv(file_path, sep="\t", index_col=0)

gene_loss_gain_x_cell_line

In [ ]:
gene_loss_gain_x_cell_line.sum(axis=1).sort_values(ascending=False)